In [1]:
# !pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
total = data_train['sales_value'].sum() # тут бы тоже надо домнажать на quantity, но ...
print(total)

# по идее, надо так, но что-то не выходит - ругается на Series name :(
# weights = data.groupby('item_id')['sales_value', 'quantity'].apply(lambda x: x['sales_value']*x['quantity'])

weighted_items = (data.groupby('item_id')['sales_value'].sum()/total).reset_index()

7052176.859999996


In [6]:
weighted_items = weighted_items.rename(columns={'sales_value': 'weight'})

In [7]:
weighted_items.head(2)

,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07


In [8]:
def weighted_random_recommendation(weighted_items, n=5):
    # спасибо коллегам за подсказку - иначе сумма вероятностей > 1 не хотела работать
    recs = weighted_items.sample(n, replace=False, weights='weight')
    
    return recs['item_id'].tolist()

In [9]:
%%time

weighted_random_recommendation(weighted_items, n=5)

CPU times: user 9.74 ms, sys: 3.63 ms, total: 13.4 ms
Wall time: 14.8 ms


[867232, 949835, 7168739, 1070702, 10341234]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [25]:
def p(v):
    return [int(id) for id in v[1:-1].split()]

result = pd.read_csv('predictions_basic.csv', converters={'actual': p, 'random_recommendation': eval, 'popular_recommendation': eval, 'itemitem': eval, 'cosine': eval, 'tfidf': eval, 'own_purchases': eval})
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [26]:
# функция из 1 вебинара
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    
    precision = flags.sum() / k
    
    return precision

In [48]:
# your_code
metrics = {}
cols = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases']

for i, r in result.iterrows():
    #print(r)
    for c in cols:
        if not c in metrics:
            metrics[c] = []
        
        #print(c, metrics[c])
        #print(r[c], r['actual'])
        metrics[c].append(precision_at_k(r[c], r['actual']))

In [51]:
mdf = pd.DataFrame(metrics)
mdf['user_id'] = result['user_id']

In [54]:
mdf.head(10)

,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,user_id
0,0.0,0.2,0.0,0.0,0.0,0.4,1
1,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,6
3,0.0,0.2,0.0,0.0,0.0,0.2,7
4,0.0,0.4,0.0,0.2,0.2,0.4,8
5,0.0,0.2,0.0,0.0,0.0,0.2,9
6,0.0,0.6,0.0,0.0,0.0,0.6,13
7,0.0,0.2,0.0,0.0,0.0,0.2,14
8,0.0,0.2,0.0,0.0,0.0,0.2,15
9,0.0,0.2,0.2,0.2,0.2,0.0,16


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

я пас - катастрофическая нехватка времени :(